In [1]:
import os

import skimage
from skimage import data
import numpy as np
from numpy import matlib
from sklearn.datasets import load_svmlight_file
from PIL import Image
import matplotlib.pylab as plt
from matplotlib.pyplot import imshow

import scfw.poisson as ps
from scfw.scopt import scopt

In [2]:
%matplotlib inline

## Data load

In [3]:
data_folder = 'data'

problem_name = 'a9a'
results = {problem_name: {}}
W, y = load_svmlight_file(os.path.join(data_folder, problem_name))
y = y.reshape(-1, 1)
W = W.toarray()

## Data preprocessing

In [4]:
# multiplicative multiplication of the structure of the recognizable structure
W = matlib.repmat(y, 1, W.shape[1]) * W
# summation of feature descriptions
sA = np.sum(W, 1)
# if the sum of the string is negative, invert it
W[sA < 0, :] = -W[sA < 0, :]
# zero row deletion
W = W[sA != 0, :]
# W = scipy.sparse.csr_matrix(W)

Bias = 1
b = (Bias * y).squeeze()
b = np.abs(b)
if (b.any == 0):
    print(' Input parameter y error')

y = b
N, n = W.shape

## Parameters initialization

In [5]:
lam =  np.sqrt(N)/2
#first set of parameters
Mf = max(2 / np.sqrt(y))
nu = 3
I = np.eye(n)
Iv = np.ones(n)
#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-10
M = ps.poisson(W, y, lam, x0)[0] / lam

FW_params={
    'iter_FW': 50000,
    'line_search_tol': 1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'estimate', #estimate L
    'use_two_phase': True,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol': 1e-2,
    'conj_grad_iter': 100,
}

## Auxilary functions

In [6]:
func_x = lambda x: ps.poisson(W, y, lam, x)
grad_x = lambda x, dot_product: ps.grad_poisson(W, y, lam, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: ps.grad_poisson(W, y, lam, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_mult_x = lambda x, dot_product: ps.hess_mult(W, y, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: ps.hess_mult_vec(W, y, s, dot_product)
extra_func = lambda x: W @ x
linear_orcale = lambda grad: ps.linear_oracle_full_simplex(grad, M)
prox_func = lambda x, L: np.maximum(x, 0) #used for SCOPT

## Run SCOPT

In [9]:
x, alpha_hist, Q_hist, time_hist = scopt(func_x,
          grad_x,
          hess_mult_x,
          hess_mult_vec_x,
          Mf,
          nu,
          prox_func,
          x0,  
          sc_params,                                              
          eps=terminate_tol,                                              
          print_every=10)
    
results[problem_name]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    #'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
}

iter =    1, stepsize = 6.186e-03, rdiff = 9.752e-02 , f = 74848

iter =   10, stepsize = 6.230e-03, rdiff = 9.568e-02 , f = 73424

iter =   20, stepsize = 6.279e-03, rdiff = 9.382e-02 , f = 71853.2

iter =   30, stepsize = 6.328e-03, rdiff = 9.237e-02 , f = 70294.9

iter =   40, stepsize = 6.384e-03, rdiff = 9.089e-02 , f = 68749.9

Fista err = 9.997e-06; Subiter = 994; subproblem converged!

Fista err = 9.995e-06; Subiter = 894; subproblem converged!

Fista err = 9.997e-06; Subiter = 910; subproblem converged!

Fista err = 9.988e-06; Subiter = 794; subproblem converged!

Fista err = 9.997e-06; Subiter = 921; subproblem converged!

Fista err = 9.994e-06; Subiter = 806; subproblem converged!

Fista err = 9.990e-06; Subiter = 910; subproblem converged!

iter =   50, stepsize = 6.445e-03, rdiff = 9.022e-02 , f = 67218.9

Fista err = 9.993e-06; Subiter = 819; subproblem converged!

Fista err = 9.994e-06; Subiter = 704; subproblem converged!

Fista err = 9.994e-06; Subiter = 792; subproble